In [ ]:
import urllib.request, urllib.error, urllib.parse
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import time

def setup_driver():
    firefox_options = Options()
    firefox_options.add_argument('--no-sandbox')
    firefox_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)
    return driver

def get_soup(url, driver):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))
    time.sleep(3)  # Wait for the dynamic content to load
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup

In [ ]:
def extract_and_write_data(soup, csv_file_path, attribute_id):
    with open(csv_file_path, 'w', encoding="utf-8", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Parameter", "Parameter Value", "Data Value"])
        main_element = soup.find('li', id=attribute_id)
        if main_element:
            facet_links = main_element.find_all('a', class_="facet_link")
            for link in facet_links:
                param_value = link['data-value']
                data_value = link.find('span', class_="item_count").text.strip('()')
                writer.writerow([attribute_id, param_value, data_value])

def main():
    url = 'https://www.macys.com/shop/womens-clothing/womens-tops/Top_style,Sortby/T-Shirt,BEST_SELLERS?id=255'
    driver = setup_driver()
    try:
        soup = get_soup(url, driver)
        # Define attributes and corresponding CSV paths
        attributes = {
            "SLEEVE_LENGTH": "macys_best_sellers_sleeve_data.csv",
            "COLOR_NORMAL": "macys_best_sellers_color_data.csv",
            "PRICE": "macys_best_sellers_price_data.csv",
            "OCCASION": "macys_best_sellers_occasion_data.csv",
            "FABRIC": "macys_best_sellers_fabric_data.csv",
            "NECKLINE": "macys_best_sellers_neckline_data.csv"
        }
        for attribute, csv_path in attributes.items():
            extract_and_write_data(soup, csv_path, attribute)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()